In [1]:
import opendatasets as od
import numpy as np
import pandas as pd

In [2]:
from keras.models import Model
import keras.backend as K
from keras.backend import mean,square,sum,shape,int_shape,random_normal,exp
from keras.layers import Input,Dense,Flatten,Conv2D,Conv2DTranspose,Lambda,Reshape,BatchNormalization,ReLU,LeakyReLU,Dropout,Activation
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

In [3]:
od.download('https://www.kaggle.com/datasets/jessicali9530/celeba-dataset')

Skipping, found downloaded files in ".\celeba-dataset" (use force=True to force download)


In [3]:
data_path = 'G:\Generative-deep-learning\Projects\celeb-face-vae\celeba-dataset\img_align_celeba'

In [4]:
img_rows,img_cols,img_chns = 256,256,3
input_dim = (img_rows,img_cols,img_chns)
batch_size = 32

: 

In [24]:
img_gen = ImageDataGenerator(rescale=1./255,data_format='channels_first')
flow = img_gen.flow_from_directory(data_path, target_size=input_dim[:2], batch_size=batch_size,shuffle=True,
                                   class_mode='input',subset='training')

Found 202599 images belonging to 1 classes.


In [11]:
# network params
input_shape = input_dim
kernel_size = 3
batch_size = 32
layers = [32,64,64,64]
# intermediate_dim = 100
latent_dim = 200


In [18]:
input_layer = Input(shape=input_shape,name='encoder_input_layer')

x = input_layer

for layer in layers:
    conv_layer = Conv2D(filters=layer, kernel_size=3,strides=2, padding='same')
    x = conv_layer(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = Dropout(rate=0.25)(x)
    

shape_before_flatten = int_shape(x)

x = Flatten()(x)

# x = Dense(intermediate_dim)(x)

mu = Dense(units=latent_dim, name = 'mu')(x)
log_var = Dense(units=latent_dim,name = 'log_var')(x)

# encoder_mu_log_var =  Model(input_layer,(mu,log_var))

# encoder_mu_log_var.summary()


In [17]:
# print(shape(mu))

In [19]:
def sampling(args):
    mu,log_var = args
    epsilon = random_normal(shape=shape(mu), mean=0, stddev=1.0)
    return mu + exp(0.5 * log_var)*epsilon

In [20]:
z = Lambda(sampling,output_shape=(latent_dim,))([mu,log_var])

encoder = Model(input_layer,[mu,log_var,z])

encoder.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input_layer (InputLaye  [(None, 128, 128, 3  0          []                               
 r)                             )]                                                                
                                                                                                  
 conv2d_4 (Conv2D)              (None, 64, 64, 32)   896         ['encoder_input_layer[0][0]']    
                                                                                                  
 batch_normalization_4 (BatchNo  (None, 64, 64, 32)  128         ['conv2d_4[0][0]']               
 rmalization)                                                                                     
                                                                                              

Total params: 1,732,944
Trainable params: 1,732,496
Non-trainable params: 448
__________________________________________________________________________________________________


In [25]:
# Decoder
decoder_temp_shape = np.prod(shape_before_flatten[1:])

decoder_input_layer = Input(shape=(latent_dim,),name='decoder_input')



x = Dense(decoder_temp_shape)(decoder_input_layer)
x = LeakyReLU()(x)
x = Reshape((128,128,64))(x)

for layer in layers[::-1]:
    conv_t_layer = Conv2DTranspose(filters=layer, kernel_size=3, strides=2, padding='same')
    x = conv_t_layer(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = Dropout(rate=0.25)(x)

decoder_output_layer = Activation('sigmoid')(x)

decoder = Model(decoder_input_layer,decoder_output_layer)

ValueError: Exception encountered when calling layer "reshape_2" (type Reshape).

total size of new array must be unchanged, input_shape = [4096], output_shape = [128, 128, 64]

Call arguments received by layer "reshape_2" (type Reshape):
  • inputs=tf.Tensor(shape=(None, 4096), dtype=float32)

In [13]:
decoder.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 decoder_input (InputLayer)  [(None, 200)]             0         
                                                                 
 dense (Dense)               (None, 4096)              823296    
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 4096)              0         
                                                                 
 reshape (Reshape)           (None, 8, 8, 64)          0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 16, 16, 64)       36928     
 nspose)                                                         
                                                                 
 batch_normalization_4 (Batc  (None, 16, 16, 64)       256       
 hNormalization)                                           

In [14]:
outs = decoder(encoder(input_layer)[2])
VAE = Model(input_layer,outs,name='VAE')
VAE.summary()

Model: "VAE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input_layer (InputL  [(None, 128, 128, 3)]    0         
 ayer)                                                           
                                                                 
 model (Functional)          [(None, 200),             1732944   
                              (None, 200),                       
                              (None, 200)]                       
                                                                 
 model_1 (Functional)        (None, 128, 128, 32)      953440    
                                                                 
Total params: 2,686,384
Trainable params: 2,685,488
Non-trainable params: 896
_________________________________________________________________


In [20]:
from keras.optimizers import RMSprop,Adam
from keras.losses import mse
import keras.backend as K
from keras.metrics import binary_crossentropy

reconstruction_loss = mse(K.flatten(input_layer),K.flatten(outs))
reconstruction_loss *= 128 * 128

kl_loss = 1 + log_var - square(mu) - exp(log_var)
kl_loss = sum(kl_loss,axis=-1)
kl_loss *= -0.5

vae_loss = mean(reconstruction_loss+kl_loss)

In [21]:
def my_vae_loss(y_true, y_pred):
    xent_loss = 128 * 128 * binary_crossentropy(K.flatten(y_true), K.flatten(y_pred))
    kl_loss = - 0.5 * K.sum(1 + log_var - K.square(mu) - K.exp(log_var), axis=-1)
    vae_loss = K.mean(xent_loss + kl_loss)
    return vae_loss

In [22]:
VAE.compile(optimizer='adam', loss = my_vae_loss)
VAE.summary()

Model: "VAE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input_layer (InputL  [(None, 128, 128, 3)]    0         
 ayer)                                                           
                                                                 
 model (Functional)          [(None, 200),             1732944   
                              (None, 200),                       
                              (None, 200)]                       
                                                                 
 model_1 (Functional)        (None, 128, 128, 32)      953440    
                                                                 
Total params: 2,686,384
Trainable params: 2,685,488
Non-trainable params: 896
_________________________________________________________________


In [ ]:
# from keras.callbacks import LearningRateScheduler
# num_epochs = 100
# learning_rate = 0.0005
# def schedule(epochs,lr,decay=0.5):
#     new_lr = lr*(decay**epochs)
#     return new_lr


# lr_scheduler = LearningRateScheduler(schedule=schedule(num_epochs,learning_rate))

In [25]:

VAE.fit(flow,shuffle = True, epochs = 100)

Epoch 1/100


TypeError: in user code:

    File "d:\condaa\envs\tflow2.9\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "d:\condaa\envs\tflow2.9\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "d:\condaa\envs\tflow2.9\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "d:\condaa\envs\tflow2.9\lib\site-packages\keras\engine\training.py", line 890, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "d:\condaa\envs\tflow2.9\lib\site-packages\keras\engine\training.py", line 948, in compute_loss
        return self.compiled_loss(
    File "d:\condaa\envs\tflow2.9\lib\site-packages\keras\engine\compile_utils.py", line 239, in __call__
        self._loss_metric.update_state(
    File "d:\condaa\envs\tflow2.9\lib